In [ ]:
include("../load.jl")

In [ ]:
include("../../data/speed_reducer.jl")

In [ ]:
gm = sagemark_to_GlobalModel(3; lse=false)
# gm = speed_reducer()
set_optimizer(gm, CPLEX_SILENT)
set_param(gm, :abstol, 1e-7)
set_param(gm, :ignore_accuracy, true)

# Actually trying to optimize...
find_bounds!(gm, all_bounds=true)
uniform_sample_and_eval!(gm)

surveysolve(gm)
println("Approximation accuracies: ", evaluate_accuracy(gm))

In [ ]:
clear_tree_constraints!(gm)
add_tree_constraints!(gm)
@time optimize!(gm)

In [ ]:
# Solving of model
set_param(gm, :ignore_accuracy, true)
# add_tree_constraints!(gm)
# optimize!(gm)
vals = solution(gm);
add_infeasibility_cuts!(gm)
optimize!(gm)
while abs(gm.cost[end] - gm.cost[end-1]) > get_param(gm, :abstol)
    add_infeasibility_cuts!(gm)
    optimize!(gm)
end
println("X values: ", vals)
println("Optimal X: ", vcat(exp.([5.01063529, 3.40119660, -0.48450710]), [-147-2/3]))

In [ ]:
gm.bbls[1].learners[end]
# gm.bbls[1].actuals
# gm.bbls[1].mi_constraints

In [ ]:
IAI.show_in_browser(gm.bbls[2].learners[end])

In [ ]:
bbr = gm.bbls[1]
score = IAI.score(bbr.learners[end], bbr.X, bbr.Y)

In [ ]:
gm.model

In [ ]:
# Printing numbers of variables and constraints
println("Variables: $(length(JuMP.all_variables(gm.model)))")
println("Constraints: $(sum(length(all_constraints(gm.model, type[1], type[2])) for type in JuMP.list_of_constraint_types(gm.model))
)")

In [ ]:
# Solving via Ipopt